## 2026 EY AI & Data Challenge - Landsat Data Extraction Notebook

This notebook demonstrates Landsat data extraction and the creation of an output file to be used by the benchmark notebook. The baseline data is [Landsat Collection 2 Level 2](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2) data from the MS Planetary Computer catalog.

**Caution**... This notebook requires significant execution time as there are 9,319 data points (unique locations and times) used for data extraction from the Landsat archive. The code takes about 7 hours to run to completion on a typical laptop computer with a typical internet connection. Lower execution times are likely possible with optimization of the data extraction process and the use of cloud computing services.


### Load In Dependencies
The following code installs the required Python libraries (found in the requirements.txt file) in the Snowflake environment to allow successful execution of the remaining notebook code. After running this code for the first time, it is required to “restart” the kernal so the Python libraries are available in the environment. This is done by selecting the “Connected” menu above the notebook (next to “Run all”) and selecting the “restart kernal” link. Subsequent runs of the notebook do not require this “restart” process. 

In [ ]:
!pip install uv
!uv pip install  -r requirements.txt 

In [1]:
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

import warnings
warnings.filterwarnings("ignore")

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

from datetime import date
from tqdm import tqdm
import os
import time
import re

tqdm.pandas()  

### Extracting Landsat Data Using API Calls

The API-based method allows us to efficiently access **Landsat** data for specific coordinates and time periods, ensuring scalability and reproducibility of the process.

Through the API, we can query individual bands or compute indices like **NDMI** on the fly. This approach reduces storage requirements and simplifies data preprocessing, making it ideal for large-scale environmental and water quality analysis.

The **compute_Landsat_values** function extracts Landsat surface reflectance values for specific sampling locations using a 100 m focal buffer around each point. For each location:

- A bounding box (bbox) is created around the latitude and longitude coordinates.
- The Microsoft Planetary Computer API is queried for Landsat-8 Level-2 surface reflectance imagery within the date range.
- The nearest low-cloud (<10% cloud cover) scene is selected, and the specified bands (**green**, **nir08**, **swir16**, **swir22**) are loaded.
- Median values of the pixels within the bounding box are computed to reduce the effect of noise or outliers.

**Why the buffer value is 0.00089831**

We want a ~100 m buffer around each point.  
At the equator, 1 degree ≈ 110 km.

Therefore, the degree equivalent of 100 m is:

*buffer_deg ≈ 100 m / 110,000 m per degree ≈ 0.00089831*

This value ensures that the buffer approximately matches the pixel resolution of Landsat imagery, capturing a ~100 m area around each sampling location.


In [2]:
# 原版的code
# # Setup
# tqdm.pandas()

# def compute_Landsat_values(row):
#     lat = row['Latitude']
#     lon = row['Longitude']
#     date = pd.to_datetime(row['Sample Date'], dayfirst=True, errors='coerce')

#     # Buffer size for ~100m 
#     bbox_size = 0.00089831  
#     bbox = [
#         lon - bbox_size / 2,
#         lat - bbox_size / 2,
#         lon + bbox_size / 2,
#         lat + bbox_size / 2
#     ]

#     catalog = pystac_client.Client.open(
#         "https://planetarycomputer.microsoft.com/api/stac/v1",
#         modifier=pc.sign_inplace,
#     )

#     # Wider search range, we'll filter to nearest date later
#     search = catalog.search(
#         collections=["landsat-c2-l2"],
#         bbox=bbox,
#         datetime="2011-01-01/2015-12-31",
#         query={"eo:cloud_cover": {"lt": 10}},
#     )
    
#     items = search.item_collection()

#     if not items:
#         return pd.Series({
#             "nir": np.nan, "green": np.nan, "swir16": np.nan, "swir22": np.nan
#         })

#     try:
#         # Convert sample date to UTC
#         sample_date_utc = date.tz_localize("UTC") if date.tzinfo is None else date.tz_convert("UTC")

#         # Pick the item closest to the sample date
#         items = sorted(
#             items,
#             key=lambda x: abs(pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc)
#         )
#         selected_item = pc.sign(items[0])

#         # Load required bands
#         bands_of_interest = ["green", "nir08", "swir16", "swir22"]
#         data = stac_load([selected_item], bands=bands_of_interest, bbox=bbox).isel(time=0)

#         green = data["green"].astype("float")
#         nir = data["nir08"].astype("float")
#         swir16 = data["swir16"].astype("float")
#         swir22 = data["swir22"].astype("float")
        
#         # Compute medians
#         median_green = float(green.median(skipna=True).values)
#         median_nir = float(nir.median(skipna=True).values)
#         median_swir16 = float(swir16.median(skipna=True).values)
#         median_swir22 = float(swir22.median(skipna=True).values)

#         # Replace 0 with NaN
#         median_green = median_green if median_green != 0 else np.nan
#         median_nir = median_nir if median_nir != 0 else np.nan
#         median_swir16 = median_swir16 if median_swir16 != 0 else np.nan
#         median_swir22 = median_swir22 if median_swir22 != 0 else np.nan
        
#         return pd.Series({
#             "nir": median_nir,
#             "green": median_green,
#             "swir16": median_swir16,
#             "swir22": median_swir22,
#         })
    
#     except Exception as e:
#         return pd.Series({
#             "nir": np.nan, "green": np.nan, "swir16": np.nan, "swir22": np.nan
#         })

### Extracting features for the training dataset

In [3]:
Water_Quality_df=pd.read_csv('water_quality_training_dataset.csv')
display(Water_Quality_df.head())

In [4]:
Water_Quality_df.shape

### Note

The Landsat data extraction process for all 9,319 locations typically requires more than 7 hours when executed in a single run. During long executions, you may occasionally encounter API limits, timeout errors, or request failures. To avoid these interruptions, we recommend running the extraction in smaller batches.

In this notebook, we provide a sample code snippet demonstrating how to extract data for the first 200 locations. Participants are encouraged to follow the same batching approach to extract data for all 9,319 locations safely and efficiently.

We have already executed the full extraction for all 9,319 locations and saved the output to **landsat_features_training.csv**, which will be used in the benchmark notebook.  
Similarly, participants can extract Landsat features in batches, combine the batch outputs, and save the final merged dataset as **landsat_features_training.csv** to ensure the benchmark notebook runs smoothly.


### API撈取Landsat的raw檔（官方資料版）

In [ ]:
def compute_Landsat_raw(row):
    lat = row['Latitude']
    lon = row['Longitude']
    date = pd.to_datetime(row['Sample Date'], dayfirst=True, errors='coerce')

    bbox_size = 0.00089831*2
    bbox = [
        lon - bbox_size / 2,
        lat - bbox_size / 2,
        lon + bbox_size / 2,
        lat + bbox_size / 2
    ]

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )

    search = catalog.search(
        collections=["landsat-c2-l2"],
        bbox=bbox,
        datetime="2011-01-01/2015-12-31",
        query={"eo:cloud_cover": {"lt": 10}},
    )

    items = search.item_collection()

    if not items:
        return pd.Series({
            "blue": None,
            "green": None,
            "red": None,
            "nir": None,
            "swir16": None,
            "swir22": None,
        })

    try:
        sample_date_utc = date.tz_localize("UTC") if date.tzinfo is None else date.tz_convert("UTC")

        items = sorted(
            items,
            key=lambda x: abs(
                pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc
            )
        )
        selected_item = pc.sign(items[0])

        bands = ["blue", "green", "red", "nir08", "swir16", "swir22"]
        data = stac_load([selected_item], bands=bands, bbox=bbox).isel(time=0)

        return pd.Series({
            "blue": data["blue"].values.flatten().tolist(),
            "green": data["green"].values.flatten().tolist(),
            "red": data["red"].values.flatten().tolist(),
            "nir": data["nir08"].values.flatten().tolist(),
            "swir16": data["swir16"].values.flatten().tolist(),
            "swir22": data["swir22"].values.flatten().tolist(),
        })

    except Exception:
        return pd.Series({
            "blue": None,
            "green": None,
            "red": None,
            "nir": None,
            "swir16": None,
            "swir22": None,
        })


def run_landsat_raw_in_batches(
    df,
    out_dir,
    batch_size,          # 先用 100~300 比較安全
    start_batch,
    pause_every,           # 每跑幾個 batch 就休息
    pause_seconds       # 休息秒數（3 分鐘）
):
    os.makedirs(out_dir, exist_ok=True)

    # 只取必要欄位並去重
    points = (
        df[['Latitude', 'Longitude', 'Sample Date']]
        .drop_duplicates()
        .reset_index(drop=True)
    )

    n = len(points)
    n_batches = (n + batch_size - 1) // batch_size

    print(f"Total unique points: {n}")
    print(f"Batch size: {batch_size}")
    print(f"Total batches: {n_batches}")
    print(f"Pause: every {pause_every} batches, sleep {pause_seconds} seconds")

    completed_this_run = 0

    for b in range(start_batch, n_batches):
        out_path = f"{out_dir}/landsat_raw_batch_{b:05d}.parquet"

        s = b * batch_size
        e = min((b + 1) * batch_size, n)
        batch_points = points.iloc[s:e].reset_index(drop=True)

        print(f"🚀 Running batch {b}/{n_batches-1}: rows {s}..{e-1} ({len(batch_points)})")

        # 呼叫 API
        batch_raw = batch_points.progress_apply(
            compute_Landsat_raw,
            axis=1
        )

        batch_df = pd.concat(
            [
                batch_points.reset_index(drop=True),
                batch_raw.reset_index(drop=True)
            ],
            axis=1
        )

        # 直接寫檔（若存在會覆蓋）
        batch_df.to_parquet(out_path, index=False)
        print(f"✅ Saved (overwritten if existed): {out_path}")

        completed_this_run += 1

        # 休息機制
        if pause_every and completed_this_run % pause_every == 0:
            print(
                f"🧘 Completed {completed_this_run} batches. "
                f"Sleeping {pause_seconds/60:.0f} minutes..."
            )
            time.sleep(pause_seconds)

    print("🎉 All batches done.")

# -----------------------------
# Merge all batch parquets -> one parquet
# -----------------------------
def merge_landsat_batches(
    out_dir,
    merged_path
):
    if not os.path.isdir(out_dir):
        raise FileNotFoundError(f"Folder not found: {out_dir}")

    # 只抓符合你 batch 命名規則的檔案：landsat_raw_batch_00000.parquet
    pattern = re.compile(r"^landsat_raw_batch_(\d{5})\.parquet$")

    batch_files = []
    for fn in os.listdir(out_dir):
        m = pattern.match(fn)
        if m:
            batch_id = int(m.group(1))
            batch_files.append((batch_id, os.path.join(out_dir, fn)))

    if not batch_files:
        raise FileNotFoundError(f"No batch parquet files found in {out_dir}")

    # 用 batch 編號排序（最穩）
    batch_files.sort(key=lambda x: x[0])
    files = [p for _, p in batch_files]

    # 讀第一個檔案當欄位基準，避免欄位不一致時 concat 默默補 NaN
    df0 = pd.read_parquet(files[0])
    base_cols = list(df0.columns)

    dfs = [df0]
    mismatched = []

    for f in files[1:]:
        dfi = pd.read_parquet(f)

        if list(dfi.columns) != base_cols:
            mismatched.append(f)
            # 對齊欄位（多的丟掉，少的補 NaN），同時提醒你有異常 batch
            dfi = dfi.reindex(columns=base_cols)

        dfs.append(dfi)

    df_all = pd.concat(dfs, ignore_index=True)
    df_all.to_parquet(merged_path, index=False)

    print(f"✅ Merged {len(files)} files -> {merged_path}")
    print("Shape:", df_all.shape)

    if mismatched:
        print(f"⚠️ Column mismatch detected in {len(mismatched)} files (aligned to base columns).")
        # 想看清單可以取消下一行註解
        # print("\n".join(mismatched))

    return df_all


run_landsat_raw_in_batches(
    df=Water_Quality_df,
    out_dir="landsat_raw_batches",
    start_batch=0,
    batch_size=100,
    pause_every=3,
    pause_seconds=180
)

landsat_raw_all = merge_landsat_batches(
    out_dir="landsat_raw_batches",
    merged_path="/tmp/landsat_raw_all.parquet"
)


### API撈取Landsat的raw檔（魔改加sample版）

In [ ]:
def compute_Landsat_raw(row):
    import numpy as np
    import pandas as pd

    lat = row['Latitude']
    lon = row['Longitude']
    date = pd.to_datetime(row['Sample Date'], dayfirst=True, errors='coerce')

    bbox_size = 0.00089831
    bbox = [
        lon - bbox_size / 2,
        lat - bbox_size / 2,
        lon + bbox_size / 2,
        lat + bbox_size / 2
    ]

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )

    # 不用 eo:cloud_cover；改用 pixel-level QA 遮罩
    search = catalog.search(
        collections=["landsat-c2-l2"],
        bbox=bbox,
        datetime="2011-01-01/2015-12-31",
    )

    items = search.item_collection()

    if not items:
        return pd.Series({
            "blue": None, "green": None, "red": None, "nir": None,
            "swir16": None, "swir22": None, "st_b10": None,
        })

    try:
        # 1) 選最近日期的 scene
        sample_date_utc = date.tz_localize("UTC") if date.tzinfo is None else date.tz_convert("UTC")
        items = sorted(
            items,
            key=lambda x: abs(pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc)
        )
        selected_item = pc.sign(items[0])

        # 2) 讀取 band（SR + ST + QA）
        bands = [
            "blue", "green", "red", "nir08", "swir16", "swir22",
            "ST_B10",
            "QA_PIXEL", "QA_RADSAT", "SR_QA_AEROSOL",
        ]

        # ✅ 關鍵：QA 用 nearest，連續值 band 可用 bilinear（或你也可全部 nearest 更保守）
        data = stac_load(
            [selected_item],
            bands=bands,
            bbox=bbox,
            resampling={
                "QA_PIXEL": "nearest",
                "QA_RADSAT": "nearest",
                "SR_QA_AEROSOL": "nearest",
                "*": "bilinear",
            }
        ).isel(time=0)

        # 3) 取出 QA（整數）
        qa_pixel = data["QA_PIXEL"].values.astype(np.uint16)
        qa_radsat = data["QA_RADSAT"].values.astype(np.uint16)
        qa_aerosol = data["SR_QA_AEROSOL"].values.astype(np.uint8)

        def bit_is_set(arr, bit):
            return (arr & (1 << bit)) != 0

        # QA_PIXEL bits: fill/dilated cloud/cirrus/cloud/shadow/snow -> 全排除
        mask_clear = (
            (~bit_is_set(qa_pixel, 0)) &
            (~bit_is_set(qa_pixel, 1)) &
            (~bit_is_set(qa_pixel, 2)) &
            (~bit_is_set(qa_pixel, 3)) &
            (~bit_is_set(qa_pixel, 4)) &
            (~bit_is_set(qa_pixel, 5))
        )

        # QA_RADSAT：任何飽和都排除
        mask_nosat = (qa_radsat == 0)

        # SR_QA_AEROSOL：bits 6-7 = aerosol level，排除 high (=3)；bit0=fill 也排除
        aerosol_level = (qa_aerosol >> 6) & 0b11
        mask_aerosol_ok = (aerosol_level != 3) & ((qa_aerosol & 1) == 0)

        mask_valid = mask_clear & mask_nosat & mask_aerosol_ok

        # 4) SR/ST 轉物理量（scale/offset）
        # SR: reflectance = DN*0.0000275 - 0.2
        # ST_B10: temp(K) = DN*0.00341802 + 149.0
        def sr_to_reflectance(da):
            dn = da.values.astype("float32")
            return dn * 0.0000275 - 0.2

        def st_to_kelvin(da):
            dn = da.values.astype("float32")
            return dn * 0.00341802 + 149.0

        blue = sr_to_reflectance(data["blue"])
        green = sr_to_reflectance(data["green"])
        red = sr_to_reflectance(data["red"])
        nir = sr_to_reflectance(data["nir08"])
        swir16 = sr_to_reflectance(data["swir16"])
        swir22 = sr_to_reflectance(data["swir22"])
        st_b10 = st_to_kelvin(data["ST_B10"])  # 先用 Kelvin；你想要 °C 我也可幫你多存一欄

        # 5) 套 mask，並「比照舊版」輸出 list
        def masked_list(arr):
            arr = arr.astype("float32")
            arr[~mask_valid] = np.nan
            return arr.flatten().tolist()

        return pd.Series({
            "blue": masked_list(blue),
            "green": masked_list(green),
            "red": masked_list(red),
            "nir": masked_list(nir),
            "swir16": masked_list(swir16),
            "swir22": masked_list(swir22),
            "st_b10": masked_list(st_b10),
        })

    except Exception:
        return pd.Series({
            "blue": None, "green": None, "red": None, "nir": None,
            "swir16": None, "swir22": None, "st_b10": None,
        })



# -----------------------------
# Merge all batch parquets -> one parquet
# -----------------------------
def merge_landsat_batches(
    out_dir,
    merged_path
):
    if not os.path.isdir(out_dir):
        raise FileNotFoundError(f"Folder not found: {out_dir}")

    # 只抓符合你 batch 命名規則的檔案：landsat_raw_batch_00000.parquet
    pattern = re.compile(r"^landsat_raw_batch_(\d{5})\.parquet$")

    batch_files = []
    for fn in os.listdir(out_dir):
        m = pattern.match(fn)
        if m:
            batch_id = int(m.group(1))
            batch_files.append((batch_id, os.path.join(out_dir, fn)))

    if not batch_files:
        raise FileNotFoundError(f"No batch parquet files found in {out_dir}")

    # 用 batch 編號排序（最穩）
    batch_files.sort(key=lambda x: x[0])
    files = [p for _, p in batch_files]

    # 讀第一個檔案當欄位基準，避免欄位不一致時 concat 默默補 NaN
    df0 = pd.read_parquet(files[0])
    base_cols = list(df0.columns)

    dfs = [df0]
    mismatched = []

    for f in files[1:]:
        dfi = pd.read_parquet(f)

        if list(dfi.columns) != base_cols:
            mismatched.append(f)
            # 對齊欄位（多的丟掉，少的補 NaN），同時提醒你有異常 batch
            dfi = dfi.reindex(columns=base_cols)

        dfs.append(dfi)

    df_all = pd.concat(dfs, ignore_index=True)
    df_all.to_parquet(merged_path, index=False)

    print(f"✅ Merged {len(files)} files -> {merged_path}")
    print("Shape:", df_all.shape)

    if mismatched:
        print(f"⚠️ Column mismatch detected in {len(mismatched)} files (aligned to base columns).")
        # 想看清單可以取消下一行註解
        # print("\n".join(mismatched))

    return df_all


run_landsat_raw_in_batches(
    df=Water_Quality_df,
    out_dir="landsat_raw_batches",
    start_batch=13,
    batch_size=100,
    pause_every=3,
    pause_seconds=5
)

landsat_raw_all = merge_landsat_batches(
    out_dir="landsat_raw_batches",
    merged_path="/tmp/landsat_raw_all.parquet"
)

### 儲存landsat的raw檔，轉存github

In [ ]:
landsat_raw_all = merge_landsat_batches(
    out_dir="landsat_raw_batches",
    merged_path="/tmp/landsat_raw_all_v2.parquet"
)

session.sql("""
    PUT file:///tmp/landsat_raw_all_v2.parquet
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

### 清理掉所有用API撈取landsat的raw檔的暫存

In [ ]:
import os

def clear_landsat_batch_dir(out_dir="landsat_raw_batches"):
    if not os.path.exists(out_dir):
        print(f"📂 Directory does not exist: {out_dir}")
        return

    files = os.listdir(out_dir)

    if not files:
        print(f"📂 Directory already empty: {out_dir}")
        return

    for f in files:
        path = os.path.join(out_dir, f)
        if os.path.isfile(path):
            os.remove(path)

    print(f"🧹 Cleared all files in: {out_dir}")
    
clear_landsat_batch_dir("landsat_raw_batches")

### 用raw檔做各種計算，以median為例

In [ ]:
path = "/tmp/landsat_raw.parquet"
landsat_raw = pd.read_parquet(path)

def compute_median_from_pixels(arr):
    # 對齊你原本邏輯：median(skipna=True) + median==0 -> NaN
    if arr is None:
        return np.nan

    a = np.array(arr, dtype="float")
    a = a[~np.isnan(a)]  # 等同 skipna=True

    if len(a) == 0:
        return np.nan

    median_val = float(np.median(a))
    return median_val if median_val != 0 else np.nan


# 1) 建立只含 key 欄位的輸出表
landsat_train_features = landsat_raw[['Latitude', 'Longitude', 'Sample Date']].copy()

# 2) 套用 median 計算（對齊你貼的四個 band）
landsat_train_features['nir']    = landsat_raw['nir'].apply(compute_median_from_pixels)
landsat_train_features['green']  = landsat_raw['green'].apply(compute_median_from_pixels)
landsat_train_features['swir16'] = landsat_raw['swir16'].apply(compute_median_from_pixels)
landsat_train_features['swir22'] = landsat_raw['swir22'].apply(compute_median_from_pixels)

# 3) 存成 parquet
landsat_train_features.to_csv("/tmp/landsat_features_training_jin.csv", index=False)


session.sql("""
    PUT file:///tmp/landsat_features_training_jin.csv
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

landsat_train_features

In [ ]:
##原本官方的code
# # Extract band values from Landsat for training dataset
# train_features_path = "landsat_features_training.csv"

# print("🚀 Running Landsat feature extraction for training data...")
# landsat_train_features = Water_Quality_df.progress_apply(compute_Landsat_values, axis=1)
# landsat_train_features.to_csv(train_features_path, index=False)

**NDMI and MNDWI Indices**

In this notebook, we compute two commonly used water-related indices from the extracted Landsat bands:

- **NDMI (Normalized Difference Moisture Index):**  
  Measures vegetation water content and surface moisture.  
  Computed as *(NIR - SWIR16) / (NIR + SWIR16)*.

- **MNDWI (Modified Normalized Difference Water Index):**  
  Highlights open water features by enhancing water reflectance and suppressing built-up areas.  
  Computed as *(Green - SWIR16) / (Green + SWIR16)*.

An **epsilon value** (*eps = 1e-10*) is added to the denominators to avoid division by zero.  
These indices are widely used in hydrological and water quality analyses for detecting water presence and vegetation moisture levels.


In [7]:
# Create indices: NDMI and MNDWI
eps = 1e-10
landsat_train_features['NDMI'] = (landsat_train_features['nir'] - landsat_train_features['swir16']) / (landsat_train_features['nir'] + landsat_train_features['swir16'] + eps)
landsat_train_features['MNDWI'] = (landsat_train_features['green'] - landsat_train_features['swir16']) / (landsat_train_features['green'] + landsat_train_features['swir16'] + eps)

In [8]:
landsat_train_features['Latitude'] = Water_Quality_df['Latitude']
landsat_train_features['Longitude'] = Water_Quality_df['Longitude']
landsat_train_features['Sample Date'] = Water_Quality_df['Sample Date']
landsat_train_features = landsat_train_features[['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']]

In [ ]:
# Preview File
landsat_train_features.head()

In [ ]:
landsat_train_features.to_csv("/tmp/landsat_features_training_20.csv",index = False)

In [ ]:
session.sql("""
    PUT file:///tmp/landsat_features_training_20.csv
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

**Note:** If you're using your own workspace, remember to replace "EY-AI-and-Data-Challenge" with your workspace name in the file path.

### Extracting features for the validation dataset

In [11]:
Validation_df=pd.read_csv('submission_template.csv')
display(Validation_df.head())

In [12]:
Validation_df.shape

In [ ]:
def run_landsat_raw_in_batches(
    df,
    out_dir,
    batch_size,          # 先用 100~300 比較安全
    start_batch,
    pause_every,           # 每跑幾個 batch 就休息
    pause_seconds       # 休息秒數（3 分鐘）
):
    os.makedirs(out_dir, exist_ok=True)

    # 只取必要欄位並去重
    points = (
        df[['Latitude', 'Longitude', 'Sample Date']]
        .drop_duplicates()
        .reset_index(drop=True)
    )

    n = len(points)
    n_batches = (n + batch_size - 1) // batch_size

    print(f"Total unique points: {n}")
    print(f"Batch size: {batch_size}")
    print(f"Total batches: {n_batches}")
    print(f"Pause: every {pause_every} batches, sleep {pause_seconds} seconds")

    completed_this_run = 0

    for b in range(start_batch, n_batches):
        out_path = f"{out_dir}/landsat_raw_batch_{b:05d}.parquet"

        s = b * batch_size
        e = min((b + 1) * batch_size, n)
        batch_points = points.iloc[s:e].reset_index(drop=True)

        print(f"🚀 Running batch {b}/{n_batches-1}: rows {s}..{e-1} ({len(batch_points)})")

        # 呼叫 API
        batch_raw = batch_points.progress_apply(
            compute_Landsat_raw,
            axis=1
        )

        batch_df = pd.concat(
            [
                batch_points.reset_index(drop=True),
                batch_raw.reset_index(drop=True)
            ],
            axis=1
        )

        # 直接寫檔（若存在會覆蓋）
        batch_df.to_parquet(out_path, index=False)
        print(f"✅ Saved (overwritten if existed): {out_path}")

        completed_this_run += 1

        # 休息機制
        if pause_every and completed_this_run % pause_every == 0:
            print(
                f"🧘 Completed {completed_this_run} batches. "
                f"Sleeping {pause_seconds/60:.0f} minutes..."
            )
            time.sleep(pause_seconds)

    print("🎉 All batches done.")


run_landsat_raw_in_batches(
    df=Validation_df,
    out_dir="landsat_validation_raw_batches",
    start_batch=0,
    batch_size=100,
    pause_every=3,
    pause_seconds=180
)

In [ ]:
import re

# -----------------------------
# Merge all batch parquets -> one parquet
# -----------------------------
def merge_landsat_batches(
    out_dir,
    merged_path
):
    if not os.path.isdir(out_dir):
        raise FileNotFoundError(f"Folder not found: {out_dir}")

    # 只抓符合你 batch 命名規則的檔案：landsat_raw_batch_00000.parquet
    pattern = re.compile(r"^landsat_raw_batch_(\d{5})\.parquet$")

    batch_files = []
    for fn in os.listdir(out_dir):
        m = pattern.match(fn)
        if m:
            batch_id = int(m.group(1))
            batch_files.append((batch_id, os.path.join(out_dir, fn)))

    if not batch_files:
        raise FileNotFoundError(f"No batch parquet files found in {out_dir}")

    # 用 batch 編號排序（最穩）
    batch_files.sort(key=lambda x: x[0])
    files = [p for _, p in batch_files]

    # 讀第一個檔案當欄位基準，避免欄位不一致時 concat 默默補 NaN
    df0 = pd.read_parquet(files[0])
    base_cols = list(df0.columns)

    dfs = [df0]
    mismatched = []

    for f in files[1:]:
        dfi = pd.read_parquet(f)

        if list(dfi.columns) != base_cols:
            mismatched.append(f)
            # 對齊欄位（多的丟掉，少的補 NaN），同時提醒你有異常 batch
            dfi = dfi.reindex(columns=base_cols)

        dfs.append(dfi)

    df_all = pd.concat(dfs, ignore_index=True)
    df_all.to_parquet(merged_path, index=False)

    print(f"✅ Merged {len(files)} files -> {merged_path}")
    print("Shape:", df_all.shape)

    if mismatched:
        print(f"⚠️ Column mismatch detected in {len(mismatched)} files (aligned to base columns).")
        # 想看清單可以取消下一行註解
        # print("\n".join(mismatched))

    return df_all

landsat_raw_all = merge_landsat_batches(
    out_dir="landsat_validation_raw_batches",
    merged_path="/tmp/landsat_validation_raw_all.parquet"
)

In [ ]:
type(landsat_val_raw.loc[10, "green"])


In [ ]:
path = "/tmp/landsat_validation_raw_all.parquet"
landsat_val_raw = pd.read_parquet(path)

def compute_median_from_pixels(arr):
    # median(skipna=True) + median==0 -> NaN
    if arr is None:
        return np.nan
    a = np.array(arr, dtype="float")
    a = a[~np.isnan(a)]
    if len(a) == 0:
        return np.nan
    median_val = float(np.median(a))
    return median_val if median_val != 0 else np.nan

# 0) 先看一下有哪些欄位（debug 用）
print("Columns:", list(landsat_val_raw.columns))

# 1) 建立輸出表：先放 key 欄位
landsat_val_features = landsat_val_raw[['Latitude', 'Longitude', 'Sample Date']].copy()

# 2) 自動對應 band 欄位名稱（避免你檔案裡不是 'nir' 而是 'nir08' 之類）
col_map_candidates = {
    "nir":    ["nir", "nir08", "B5", "SR_B5"],
    "green":  ["green", "B3", "SR_B3"],
    "swir16": ["swir16", "swir1", "B6", "SR_B6"],
    "swir22": ["swir22", "swir2", "B7", "SR_B7"],
}

resolved = {}
for out_name, candidates in col_map_candidates.items():
    found = next((c for c in candidates if c in landsat_val_raw.columns), None)
    resolved[out_name] = found

missing = [k for k, v in resolved.items() if v is None]
if missing:
    raise KeyError(
        f"These expected bands are missing in parquet: {missing}. "
        f"Available columns: {list(landsat_val_raw.columns)}"
    )

print("Resolved band columns:", resolved)

# 3) median 計算：從 landsat_val_raw 的原始像素陣列欄位算出來，寫入 features
for out_name, raw_col in resolved.items():
    landsat_val_features[out_name] = landsat_val_raw[raw_col].apply(compute_median_from_pixels)

# 5) PUT 到 Snowflake workspace
session.sql(f"""
    PUT file:///tmp/landsat_validation_raw_all.parquet
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")


In [14]:
# Create indices: NDMI and MNDWI
eps = 1e-10
landsat_val_features['NDMI'] = (landsat_val_features['nir'] - landsat_val_features['swir16']) / (landsat_val_features['nir'] + landsat_val_features['swir16'])
landsat_val_features['MNDWI'] = (landsat_val_features['green'] - landsat_val_features['swir16']) / (landsat_val_features['green'] + landsat_val_features['swir16'] + eps)

In [15]:
landsat_val_features['Latitude'] = Validation_df['Latitude']
landsat_val_features['Longitude'] = Validation_df['Longitude']
landsat_val_features['Sample Date'] = Validation_df['Sample Date']
landsat_val_features = landsat_val_features[['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']]

In [ ]:
# Preview File
landsat_val_features.head()
landsat_val_features.to_csv("/tmp/landsat_features_validation_jin.csv",index = False)

In [ ]:
session.sql("""
    PUT file:///tmp/landsat_features_validation_jin.csv
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

**Note:** If you're using your own workspace, remember to replace "EY-AI-and-Data-Challenge" with your workspace name in the file path.